In [1]:
import pandas as pd
import numpy as np
import datetime
import random

In [2]:
%ls

app/               items.csv                 README.md
bill_of_lading.py  notes_about_csv_file.txt  requirements.txt
config.py          __pycache__/              shipment_builder.ipynb
instance/          QUESTION.docx             shipment_requirements.yml


In [3]:
with open("notes_about_csv_file.txt") as notes:
    print(notes.read())

The items.csv file is structured as such:

item_id	  item_group  cubic_volume_ft
10413	  A	          0.1
10341	  A	          0.5
10004	  B	          1.0
80014	  C	          0.3
20242	  B	          0.4
…	      …	          …

Each record in this csv file is representative of a single item.
The item_id field is a unique identifier for the item, while the item_type and cubic_volume_ft fields are attributes of the item.



Clean_csv will need to have it's path changed once data/tmp/ is being implemented and build.items() used

In [4]:
def clean_csv():
    import pandas as pd

    stock = pd.read_csv("app/data/items.csv")

    # Isolate the data to only the three necessary columns:
        # 'item_id', 'item_group', 'cubic_volume_ft'
    # Drop any NaN rows from the data
  
    return  (stock.loc[:,['item_id', 'item_group', 'cubic_volume_ft']]
                  .dropna()
            )
    
    '''
    Future implementation will include functionality for: 
        Handling NaN values beyond just dropping them
        Include column testing to ensure data types
    '''

Clean should be renamed to extract_csv and then a pipeline module build. pipline.extract_csv, pipeline.transform() which will include any DataFrame transformations that need to happen prior to data processing (namely sorting)

In [5]:
items = clean_csv().sort_values("cubic_volume_ft",
                                ascending=True)

In [6]:
items.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 200 entries, 161 to 81
Data columns (total 3 columns):
item_id            200 non-null float64
item_group         200 non-null object
cubic_volume_ft    200 non-null float64
dtypes: float64(2), object(1)
memory usage: 6.2+ KB


In [7]:
items.head()

,item_id,item_group,cubic_volume_ft
161,8178.0,C,0.11
179,9667.0,B,0.11
163,4093.0,C,0.12
135,1367.0,D,0.12
56,2194.0,C,0.13


In [8]:
def dataframe_generator():
    import random
    import pandas as pd
    return (pd.DataFrame(data = {'key': [random.randint(0,199) for x in range(random.randint(175,225))]})
              .merge(clean_csv(), 
                     left_on='key', 
                     right_index=True)
              .drop('key',
                    axis = 1)
              .reset_index(drop=True)
           )

In [9]:
def csv_generator():    
    # Builds anywhere from 150-250 rows of data that matches what comes from items.csv
    # Creates random 'keys' that it merges with the stock data based on index
    # returns the data as .csv's
    
    import random
    import glob
    import re
    
    count = sorted([int(re.sub("[^0-9]", "", files)) for files in glob.glob("app/data/tmp/*.csv")])[-1] + 1
    if not count: 
        count = '1'        
    
    return (pd.DataFrame(data = {'key': [random.randint(0,199) for x in range(random.randint(150,250))]})
              .merge(clean_csv(), 
                     left_on='key', 
                     right_index=True)
              .drop('key',
                    axis = 1)
              .reset_index(drop=True)
           ).to_csv("app/data/tmp/items"+str(count)+".csv", 
                    index=False)

In [10]:
def stock():
    import glob
    
    # It's nice to assume clean data, and to be right for once
    
    stock = pd.DataFrame()
    for csv in glob.glob("app/data/tmp/*.csv"):
        stock = stock.append(pd.read_csv(csv))
    
    return stock.reset_index(drop=True).sort_values('cubic_volume_ft')

In [11]:
def generate_shipment_id():
    import re
    return int(re.sub("[^0-9]", "", str(datetime.datetime.today()))[:17])

Future improvements will use arrays of idx and vol^3 zipped together for speed improvements.

The result will be a dictionary of shipment_id and idx this will be able to be merged to result in final shipment.

This implementation will have to be tested to prove speed improvements exist from this.

In [52]:
def shipments(stock) :
    # Create a blank shipment sheet
    shipments = {}
    
    while stock.empty == False :

        # Get the largest item by cubic volume and remove from stock
        bundle, stock = stock.tail(1), stock.drop(stock.tail(1).index, axis=0)
        
        # Filter the remaining stock by what CAN still fit in the box
        # Grab the index of the item and the item
        for index, item in (stock[stock.cubic_volume_ft.values < (1.58 - bundle.cubic_volume_ft.values)]
                            .sort_values("cubic_volume_ft",
                                         ascending=False)
                           ).iterrows():
            
            # If there is no item in stock that could fit into the bundle break out of the matrix
            if (bundle.cubic_volume_ft.sum() + stock.cubic_volume_ft.values.min()) > 1.58 :
                break
                
            # If it fits it sits
            # Add the item to the bundle
            # Drop item from the stock
            elif (bundle.cubic_volume_ft.sum() + item.cubic_volume_ft) <= 1.58 :
                item, stock = (item, stock.drop(index))
                bundle = bundle.append(item)
                
        #Issue a shipment id to the bundle
        shipments[generate_shipment_id()] = bundle

    return shipments

SyntaxError: invalid syntax (<ipython-input-52-73364542b370>, line 9)

In [14]:
items.head()

,item_id,item_group,cubic_volume_ft
161,8178.0,C,0.11
179,9667.0,B,0.11
163,4093.0,C,0.12
135,1367.0,D,0.12
56,2194.0,C,0.13


In [53]:
shipments(items)

KeyError: 0

In [40]:
shipment.head()

item_id item_group  cubic_volume_ft
20190304180607610 81    4643.0          B             1.30
                  154   8629.0          C             0.27
20190304180607622 189   6549.0          A             1.29
                  69    2207.0          B             0.29
20190304180607635 6     4235.0          B             1.28

In [90]:
len(shipment.index.get_level_values(0).unique())

87

In [99]:
def summary(shipments):
    
    # Build initial summaries based on items and cubic volume in feet
    data = {'Total Items' : len(shipments.item_id.values),
            'Total Cubic Volume in Feet' : shipments.cubic_volume_ft.values.sum(),
            'Total Item Groups' : len(shipments.item_group.unique())}
    
    # Check for shipment id and build additional shipment summaries
    if shipment.index.get_level_values(0).any() :
        shipment_id = shipment.index.get_level_values(0).unique()
        data['Total Shipments'] = len(shipment_id)
        data['Shipment Item Ratio'] = round(len(shipments.item_id.values) / len(shipment_id),2)
        data['Cubic Volume not Utilized'] = (1.58*len(shipment_id) - 
                                             shipments.cubic_volume_ft.values.sum())
        data['Percent Cubic Volume not Utilized'] = round(((1.58 * len(shipment_id) - 
                                                            shipments.cubic_volume_ft.values.sum()) / 
                                                     shipments.cubic_volume_ft.values.sum()) * 100, 2)
    # return resulting summary as a DataFrame
    return (pd.DataFrame(data, 
                         index=['Details'])
           )

In [100]:
summaries = summary(shipment)

In [101]:
summaries['Total Shipments'][0]

87

In [102]:
summaries.head()

,Total Items,Total Cubic Volume in Feet,Total Item Groups,Total Shipments,Shipment Item Ratio,Cubic Volume not Utilized,Percent Cubic Volume not Utilized
Details,200,135.52,4,87,2.3,1.94,1.43


## Grouping Prototypes

In [ ]:
def get_groups(items):
    if 'item_group' in items.keys() :
        return items.item_group.unique()
    else :
        return None

In [ ]:
shipments_filtered = {}
for group in get_groups(items):
    stock_filtered = items[items.item_group.values == group]
    shipments_filtered[group] = shipments(stock_filtered)

In [ ]:
shipments_filtered['A'].head()